### classify stackoverflow posts using tf-idf features, using neural nets

- note: use the same tokenizer rules as Keras

In [10]:
import csv
import os
import re
import sys
import numpy as np
import tensorflow as tf

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Dropout, Input, Flatten, Activation, Lambda
from keras.layers import Conv1D, MaxPooling1D, Embedding, GlobalAvgPool1D
from keras.models import Model, Sequential

from sklearn import preprocessing, svm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import cross_val_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from sklearn.pipeline import Pipeline

# my stuff in the helpers/ directory
from helpers import files_helper, texts_helper, metrics_helper, tags_helper

In [2]:
SEED=np.random.randint(1,1000)
SEED

520

In [3]:
np.random.seed(seed=SEED)

In [4]:
texts,labels_index,labels_0 = files_helper.read_20_newgroup_files()       

In [5]:
MAX_NB_WORDS = 20000
MAX_SEQUENCE_LENGTH = 1000
VALIDATION_SPLIT = 0.2
EMBEDDING_DIM=100
LABELS_MIN_DOC_COUNT = int(10)
BATCH_SIZE=32
NUM_EPOCHS=10

In [6]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS,
                     filters='\'!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index
inverse_word_index = texts_helper.build_inverse_word_index(word_index)

In [7]:
[inverse_word_index[idx] for idx in sequences[0]]

['archive',
 'name',
 'atheism',
 'resources',
 'alt',
 'atheism',
 'archive',
 'name',
 'resources',
 'last',
 'modified',
 '11',
 'december',
 '1992',
 'version',
 '1',
 '0',
 'atheist',
 'resources',
 'addresses',
 'of',
 'atheist',
 'organizations',
 'usa',
 'freedom',
 'from',
 'religion',
 'foundation',
 'darwin',
 'fish',
 'bumper',
 'stickers',
 'and',
 'assorted',
 'other',
 'atheist',
 'are',
 'available',
 'from',
 'the',
 'freedom',
 'from',
 'religion',
 'foundation',
 'in',
 'the',
 'us',
 'write',
 'to',
 'p',
 'o',
 'box',
 '750',
 'madison',
 'wi',
 'telephone',
 '608',
 '256',
 '8900',
 'evolution',
 'designs',
 'evolution',
 'designs',
 'sell',
 'the',
 'darwin',
 'fish',
 'it',
 's',
 'a',
 'fish',
 'symbol',
 'like',
 'the',
 'ones',
 'christians',
 'stick',
 'on',
 'their',
 'cars',
 'but',
 'with',
 'feet',
 'and',
 'the',
 'word',
 'darwin',
 'written',
 'inside',
 'the',
 'deluxe',
 '3d',
 'plastic',
 'fish',
 'is',
 '4',
 '95',
 'in',
 'the',
 'us',
 'write',


In [8]:
tokenized_texts = []

for seq in sequences:
    truncated_seq = seq[:MAX_SEQUENCE_LENGTH]
    tokenized_txt = " ".join([inverse_word_index[idx] for idx in truncated_seq])
    tokenized_texts.append(tokenized_txt)

In [33]:
data = tokenized_texts
indices = np.arange(len(data))
np.random.shuffle(indices)

labels_1 = to_categorical(np.asarray(labels_0))

data = [data[i] for i in indices]
labels_2 = labels_1[indices]
num_validation_samples = int(VALIDATION_SPLIT * len(data))

data_train = data[:-num_validation_samples]
data_val = data[-num_validation_samples:]


In [34]:
# TfidfVectorizer = CountVectorizer + TFidfTransformer

# does this configure snooping?
vect = TfidfVectorizer(max_features=MAX_NB_WORDS).fit(data)
X_train = vect.transform(data_train)
X_val = vect.transform(data_val)

Y_train = labels_2[:-num_validation_samples]
Y_val = labels_2[-num_validation_samples:]

X_train = X_train.todense()
X_val = X_val.todense()

X_train.shape,X_val.shape,Y_train.shape,Y_val.shape

((15998, 19859), (3999, 19859), (15998, 20), (3999, 20))

In [26]:
X_train.shape[1]

19859

In [35]:
model = Sequential()

model.add(Dense(64,activation='relu',input_dim=X_train.shape[1]))
model.add(Dropout(0.5))
model.add(Dense(64,activation='sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(Y_train.shape[1], activation='softmax'))

model.compile(loss ='categorical_crossentropy',
             optimizer='adam',
             metrics=['acc'])

In [36]:
%%time

model.fit(X_train,Y_train, validation_data=(X_val, Y_val),
         epochs=NUM_EPOCHS, batch_size=BATCH_SIZE)

Train on 15998 samples, validate on 3999 samples
Epoch 1/10
15998/15998 [==============================] - 7s - loss: 2.8771 - acc: 0.1206 - val_loss: 2.2624 - val_acc: 0.5256
Epoch 2/10
15998/15998 [==============================] - 7s - loss: 1.9958 - acc: 0.3741 - val_loss: 1.4054 - val_acc: 0.7314
Epoch 3/10
15998/15998 [==============================] - 7s - loss: 1.4131 - acc: 0.5623 - val_loss: 0.9728 - val_acc: 0.7967
Epoch 4/10
15998/15998 [==============================] - 7s - loss: 1.0811 - acc: 0.6658 - val_loss: 0.7510 - val_acc: 0.8172
Epoch 5/10
15998/15998 [==============================] - 7s - loss: 0.8665 - acc: 0.7333 - val_loss: 0.6248 - val_acc: 0.8325
Epoch 6/10
15998/15998 [==============================] - 7s - loss: 0.7148 - acc: 0.7818 - val_loss: 0.5533 - val_acc: 0.8372
Epoch 7/10
15998/15998 [==============================] - 7s - loss: 0.6049 - acc: 0.8147 - val_loss: 0.5154 - val_acc: 0.8422
Epoch 8/10
15998/15998 [==============================] - 7s -